In [1]:
!pip install openpyxl

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn
from dateutil.relativedelta import relativedelta
import os
import glob
import math
import boto3
import s3fs
import tempfile
import joblib
# import h5py
from config_module import Config
config = Config()

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from statsmodels.tsa.stattools import adfuller
import itertools
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
# Get the current datetime
current_datetime = datetime.now()

In [3]:
fs = s3fs.S3FileSystem()
s3_client = boto3.client('s3')

In [4]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.xlsx':
        # Read Excel file
        data = pd.read_excel(file_path)
    elif file_extension == '.csv':
        # Read CSV file
        data = pd.read_csv(file_path)
    elif file_extension == '.txt':
        # Read TXT file
        data = pd.read_csv(file_path)
    else:
        raise ValueError("Unsupported file format: {}".format(file_extension))
    
    return data

def calculate_mape(x,y):
    import math
    if x==y:
        mape=1
    elif x==0:
        mape=0
    elif math.isnan(x):
        mape=0
    else:
        mape=1-(abs(x-y)/x)
    return mape

In [5]:
def normalize_forecast(df_forecast, colname):
    df_forecast[colname] = df_forecast[colname].apply(lambda x:int(round(x)))
    df_forecast[colname] = df_forecast[colname].apply(lambda x: np.where(x<0,max(x,0),x))
    return df_forecast

In [6]:
def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    if 'Contents' not in response:
        print("No files found.")
        return None
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")
 
    if not files_with_dates:
        print("No valid files with dates found.")
        return None
 
    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    return latest_file_key

In [7]:
bucket = config.get_value('bucket')
bucket_new = config.get_value('bucket_new')
prefix_output_file = config.get_value('prefix_cleaned_training_file_path')
file_name_prefix = config.get_value('cleaned_training_filename_prefix')
file_path = get_latest_s3_file(bucket,prefix_output_file,file_name_prefix)

data_old_lag1 = pd.read_excel('s3://'+f'{bucket}/{file_path}')
parts_list = data_old_lag1['part_number'].unique().tolist()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [8]:
data_old_lag1 = data_old_lag1.sort_values(['part_number', 'period'])

In [9]:
def calculate_mape(x, y):
    """
    Function to calculate the Mean Absolute Percentage Error (MAPE) between two values x and y.
    
    Parameters:
    x: The true value.
    y: The predicted value.
    
    Returns:
    The MAPE value as a float.
    """
    
    # Check if x is NaN
    if math.isnan(x):
        return 0
    
    # Check if x and y are equal
    if x == y:
        return 1
    
    # Check if x is zero
    elif x == 0:
        return 0
    
    # Calculate MAPE
    else:
        mape = 1 - (abs(x - y) / x)
        return mape

def find_outliers_iqr_new(data, column_name):
    out_data = pd.DataFrame()
    for part in data['part_number'].unique().tolist():
        df = data[data['part_number']==part]
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[column_name].quantile(0.25)
        Q3 = df[column_name].quantile(0.75)
        # Calculate the IQR (Interquartile Range)
        IQR = Q3 - Q1
        # Determine the outlier cutoffs
        upper_bound = Q3 + 1.5 * IQR
        # Identify outliers
        df_treated = df.copy()
        df_treated['olt_cnt'] = [np.nan]*len(df_treated)
        df_treated.loc[df_treated[column_name]>upper_bound,'olt_flg'] = 1
        df_treated['olt_cnt'] = df_treated['olt_flg'].groupby(df_treated['part_number']).transform('sum')
        out_data = pd.concat([out_data,df_treated],ignore_index=True)
    out_data['olt_flg'] = out_data['olt_flg'].fillna(0)    
    return out_data

In [10]:
def create_threshold(acc, var):
        if acc >= 0.8:
            return True
        elif acc<0.8 and var==1:
            return True
        else:
            return False

def replace_sales_missing(row):
    if pd.isna(row['Total_qty_per_month']):
        if pd.notna(row['last_6_months_avg']):
            return row['last_6_months_avg']
        elif pd.notna(row['last_3_months_avg']):
            return row['last_3_months_avg']
        else:
            if row['olt_cnt'] > 0:
                return row['sales_median']
            else:
                return row['sales_mean']
    else:
        return row['Total_qty_per_month']


def replace_outliers(row):
    if row['olt_flg']>0:
        if pd.notna(row['last_6_months_avg']):
            return row['last_6_months_avg']
        elif pd.notna(row['last_3_months_avg']):
            return row['last_3_months_avg']
        else:
            if row['olt_flg'] > 0:
                return row['sales_median']
    else:
        return row['Total_qty_per_month']


def impute_missing_sales(df):
    # Sort DataFrame by part_number and date
    df.sort_values(['part_number', 'period'], inplace=True)
 
    def rolling_avg(row, window):
        part_data = df[(df['part_number'] == row['part_number']) & (df['period'] < row['period'])& (df['olt_flg']==0) &
                    pd.notna(df['Total_qty_per_month'])]
        if len(part_data) >= window:
            return part_data.tail(window)['Total_qty_per_month'].mean()
        else:
            return None
 
    # Calculate last 6 months average sales considering missing values
    df['last_6_months_avg'] = df.apply(lambda row: rolling_avg(row, 6), axis=1)
   
    # Calculate last 3 months average sales considering missing values
    df['last_3_months_avg'] = df.apply(lambda row: rolling_avg(row, 3), axis=1)
   
    # Calculate mean and median for each part
    df_agg = df.groupby('part_number')['Total_qty_per_month'].agg(['median', 'mean']).reset_index()
    df_agg.columns = ['part_number', 'sales_median', 'sales_mean']
 
    # Merge aggregated data with the main DataFrame
    df = pd.merge(df, df_agg, on='part_number', how='left')
 
    # Apply replacement function
    df['Total_qty_per_month'] = df.apply(replace_sales_missing, axis=1)
   
    return df

def impute_outlier_sales(df):
    
    # Sort DataFrame by part_number and date
    df.sort_values(['part_number', 'period'], inplace=True)
   
    # Initialize consecutive_non_missing count
    def rolling_avg(row, window):
        part_data = df[(df['part_number'] == row['part_number']) & (df['period'] < row['period']) & (df['olt_flg']==0) &
                    pd.notna(df['Total_qty_per_month'])]
        if len(part_data) >= window:
            return part_data.tail(window)['Total_qty_per_month'].mean()
        else:
            return None
   
    # Calculate last 6 months average sales considering missing values
    df['last_6_months_avg'] = df.apply(lambda row: rolling_avg(row, 6), axis=1)
   
    # Calculate last 3 months average sales considering missing values
    df['last_3_months_avg'] = df.apply(lambda row: rolling_avg(row, 3), axis=1)
   
    # Calculate mean and median for each part
    df_agg = df.groupby('part_number')['Total_qty_per_month'].agg(['median', 'mean']).reset_index()
    df_agg.columns = ['part_number', 'sales_median', 'sales_mean']
 
    # Merge aggregated data with the main DataFrame
    df = pd.merge(df, df_agg, on='part_number', how='left')
 
    # Apply replacement function
    df['Total_qty_per_month'] = df.apply(replace_outliers, axis=1)
   
    return df

def missing_val_df(df_train,max_tr_date):
# Group by part number and calculate min and max dates for each part
    min_max_dates = df_train.groupby('part_number')['period'].agg(['min'])
    min_max_dates['max'] = pd.to_datetime(max_tr_date)
   
 
    # Reindex the DataFrame with all months for each part
    dfs = []
    for part, (min_date, max_date) in min_max_dates.iterrows():
        date_range = pd.date_range(start=min_date, end=max_date, freq='MS')
        part_df = pd.DataFrame({'period': date_range})
        part_df['part_number'] = part
        dfs.append(part_df)
 
    df_all = pd.concat(dfs)
 
    # Merge with original data to retain sales values
    df_all = pd.merge(df_all, df_train, on=['part_number', 'period'], how='left')
 
    df_all['olt_cnt'] = df_all['olt_cnt'].fillna(method = 'bfill')
    df_all['olt_flg'] = df_all['olt_flg'].fillna(0)
 
    # Replace missing sales values based on outlier count
    df_train_missing_treated = impute_missing_sales(df_all)
    df_train_missing_treated=df_train_missing_treated.drop(['last_6_months_avg', 'last_3_months_avg', 'sales_median', 'sales_mean'],axis=1)
    return df_train_missing_treated

# Group by part number and calculate min and max dates for each part
def oltlier_df(df_train):
    df_train_missing_olt_treated = impute_outlier_sales(df_train)
    df_train_missing_olt_treated=df_train_missing_olt_treated.drop(['last_6_months_avg', 'last_3_months_avg', 'sales_median', 'sales_mean'],axis=1)
    return df_train_missing_olt_treated

def create_test_complete_dataset_from_scratch(df, start_date, end_date, pn):
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
     
    # Create a DataFrame with all months in the specified period
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    all_product_df = pd.DataFrame({'part_number':[pn]})
    
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    return merged_df

def create_test_complete_dataset(df, start_date, end_date):
    # if range_val == 3:
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
     
    # Create a DataFrame with all months in the specified period
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    all_months_df = pd.DataFrame(date_range, columns=['period'])
    all_months_df['period'] = all_months_df['period'].dt.strftime('%Y-%m')
    all_product_df = pd.DataFrame({'part_number':df['part_number'].unique()})
     
    merged_df = all_months_df.merge(all_product_df, on=None, how='cross')
    merged_df['period'] = pd.to_datetime(merged_df['period'])
    merged_df = merged_df.merge(df, left_on=['period','part_number'],
                                right_on=['period','part_number'], how='left')

    return merged_df

def create_complete_dataset(data_new, train_start_date, train_end_date):
    data_new['period'] = pd.to_datetime(data_new['period'])
    data_new['period'] = data_new['period'].dt.strftime('%Y-%m-%d')
    data_new = data_new.sort_values(['part_number', 'period'])
    data_new = data_new[(data_new['period']>=train_start_date)]
    data_new['period'] = pd.to_datetime(data_new['period'])

    df_filtered_1=data_new.copy()
    df_train = df_filtered_1[df_filtered_1['period']<=(train_end_date)]
    

    test_start_date = pd.to_datetime(pd.to_datetime(train_end_date)+pd.DateOffset(months=1)).strftime('%Y-%m-%d')
    test_end_date = pd.to_datetime(pd.to_datetime(train_end_date)+pd.DateOffset(months=3)).strftime('%Y-%m-%d')
    df_test = df_filtered_1[(df_filtered_1['period']>=(test_start_date))&(df_filtered_1['period']<=(test_end_date))]
    return df_train, df_test

def get_consecutive_months(start_date, n_months=3):
    """
    Generate a list of consecutive month start dates.

    Args:
    - start_date (str): Starting date in 'YYYY-MM-DD' format.
    - n_months (int): Number of consecutive months to generate.

    Returns:
    - list: List of consecutive month start dates in 'YYYY-MM-DD' format.
    """
    start_date = pd.to_datetime(start_date)
    month_list = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, n_months + 1)]
    return month_list

def get_latest_s3_file(bucket_name, path_prefix, file_prefix):
    s3 = boto3.client('s3')
    
    # List all objects under the given path_prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path_prefix)
    
    if 'Contents' not in response:
        print("No files found.")
        return None
    
    # Filter objects that start with the specific file path_prefix
    filtered_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].startswith(f"{path_prefix}{file_prefix}")
    ]
    
    if not filtered_files:
        print("No files found with the specific path_prefix.")
        return None
    
    # Extract the date from each file name and store along with the key
    files_with_dates = []
    for obj in filtered_files:
        key = obj['Key']
        # Assuming the date is at the end of the filename after the last underscore
        try:
            # Get the base name without directory path
            base_name = os.path.basename(key)
            name_without_extension = os.path.splitext(base_name)[0]
            date_str = name_without_extension.split('_')[-1]  # Get the date string part
            file_date = datetime.strptime(date_str, "%Y-%m-%d")  # Parse date
            files_with_dates.append((key, file_date))
        except ValueError as e:
            print(f"Error parsing date from file name {key}: {e}")

    if not files_with_dates:
        print("No valid files with dates found.")
        return None

    # Find the file with the latest date
    latest_file_key, latest_date = max(files_with_dates, key=lambda x: x[1])
    
    return latest_file_key

#### Setting up all negative forecasted parts for run

In [11]:
prefix_bestfit_trained_data = config.get_value('prefix_bestfit_file')
# prefix_bestfit_trained_data = prefix_bestfit_trained_data +'/'
bestfit_trained_data_filename = config.get_value("file_name_bestfit_powerbi")
latest_bestfit_trained_data_path = get_latest_s3_file(bucket_new, prefix_bestfit_trained_data+"/", bestfit_trained_data_filename)
print(latest_bestfit_trained_data_path)
trained_bestfit_data = pd.read_excel('s3://'+f'{bucket_new}/{latest_bestfit_trained_data_path}')

curated-bo/curated-bo-demand-forecast/bestfit-selected-models/bestfit_selected_models_historic_months_powerbi_2024-11-03.xlsx


In [12]:
prefix_forecasted_data = config.get_value('prefix_forecast_file')
bestfit_forecasted_data_filename = config.get_value('filename_forecast_powerbi')
latest_bestfit_forecasted_data_path = get_latest_s3_file(bucket_new, prefix_forecasted_data, bestfit_forecasted_data_filename)
print(latest_bestfit_forecasted_data_path)
forecasted_bestfit_data = pd.read_excel('s3://'+f'{bucket_new}/{latest_bestfit_forecasted_data_path}')

curated-bo/curated-bo-demand-forecast/selected-models-forecasted-output/forecast_selected_models_future_months_powerbi_2024-11-03.xlsx


In [13]:
total_powerbi_data = pd.concat([trained_bestfit_data, forecasted_bestfit_data], ignore_index=True)

In [14]:
negative_parts = total_powerbi_data[total_powerbi_data['Forecasted']<0]['Part Number'].unique().tolist()

In [15]:
len(negative_parts)

160

### ARIMA Training

In [16]:
def arima_model(data_ts, test_data, p_range, d_range, q_range, pn, train_x, valid_x):
    final_arima_df = pd.DataFrame()
    internal_arima_dict = pd.DataFrame()
    final_df = pd.DataFrame()
    model_list = []
    for p in range(0, p_range):  # AR parameter
        for d in range(0, d_range):  # Differencing parameter
            for q in range(0, q_range):  # MA parameter
                model = ARIMA(data_ts, order=(p, d, q))
                try: 
                    fitted_model = model.fit()
                    forecast_valid = fitted_model.forecast(steps=3)
                    forecast_valid = forecast_valid.tolist()
                    forecast_valid = list(map(lambda x: round(x, 2), forecast_valid))
                    fitted_values = fitted_model.fittedvalues.tolist()
                    fitted_values = list(map(lambda x: round(x, 2), fitted_values))
                    model_name = f"ARIMA({p},{d},{q})"
                    final_arima_df_test = pd.DataFrame({
                        'part_number': [pn]*test_data.shape[0],
                        'period':valid_x.values.flatten(),
                        'type':['test']*test_data.shape[0],
                        'actual':test_data.values,
                        'fitted':forecast_valid,
                        'model_name':[model_name]*test_data.shape[0]})
                    final_arima_df_train = pd.DataFrame({
                        'part_number': [pn]*data_ts.shape[0],
                        'period':train_x.values.flatten(),
                        'type':['train']*data_ts.shape[0],
                        'actual':data_ts.values,
                        'fitted':fitted_values,
                        'model_name':[model_name]*data_ts.shape[0]})
                    final_df = pd.concat([final_df, final_arima_df_train, final_arima_df_test], ignore_index=True)
                except:
                    continue

    return final_df

In [17]:
def create_TS_model_arima(df, p_range, d_range, q_range, t_start_date, t_end_date, lag_val):
    
    # Split data into training and testing sets
    pn = df['part_number'].unique().tolist()[0]
    df_new = df[:]

    train, test = create_complete_dataset(df_new, t_start_date, t_end_date)
    train_clean = train[:]
    if test.empty:
        print(pn)
        test = create_test_complete_dataset_from_scratch(test, 
                                                         train_clean['period'].max()+pd.DateOffset(months=1), 
                                                         train_clean['period'].max()+pd.DateOffset(months=3),
                                                         pn)
    if (test.shape[0]<3):
        test = create_test_complete_dataset(test, 
                                            train_clean['period'].max()+pd.DateOffset(months=1), 
                                            train_clean['period'].max()+pd.DateOffset(months=3))
    data_start_date = train['period'].min()
    train_end_date = train['period'].max()
    test_start_date = test['period'].min()
    data_end_date = test['period'].max()
    df_new['period'] = pd.to_datetime(df_new['period'])
    print("Total data date::", data_start_date, train_end_date, test_start_date, data_end_date)
    period_data = pd.concat([train_clean, test], ignore_index=True)
    arima_all_df = arima_model(train['Total_qty_per_month'][:], test['Total_qty_per_month'][:], p_range, 
                                               d_range, q_range, pn, train['period'], test['period'])
    return arima_all_df

In [18]:
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore')

In [19]:
print(datetime.now())

2024-11-05 12:58:05.577676


In [20]:
training_end_date = (pd.to_datetime(data_old_lag1['period']).max() + relativedelta(months=-3)).strftime('%Y-%m-%d')
training_start_date = (pd.to_datetime(data_old_lag1['period']).min()).strftime('%Y-%m-%d')

In [21]:
def arima_training(parts_list_updated, train_data, start_date_train, end_date_train):
    ind = 0
    final_best_df = pd.DataFrame()
    # index_val=0
    arima_data = pd.DataFrame()
    for parts in parts_list_updated:

        print(parts, ind)

        print('-------------------Start-----------------------')

        for lag_val in range(1):
            if lag_val==0:
                print(1)
                parts_data_new = train_data[train_data['part_number']==parts].drop_duplicates()
                arima_df = create_TS_model_arima(parts_data_new, 5,2,5, start_date_train, end_date_train, lag_val)
                arima_data = pd.concat([arima_data, arima_df], ignore_index=True)
                pivot_df = arima_df.pivot(index='period', 
                                          columns='model_name', 
                                          values='fitted')
                merged_df_arima = arima_df.drop(['fitted', 
                                                 'model_name'], axis=1).drop_duplicates().merge(pivot_df, 
                                                                                                    how='left', 
                                                                                                    on='period')
                final_best_df = pd.concat([final_best_df, merged_df_arima], ignore_index=True)
        ind+=1
        print('-------------------End-----------------------')
    
#     Training concludes
    
    arima_data = arima_data[arima_data['model_name']!='ARIMA(0,0,0)'].reset_index(drop=True)
    final_best_df.drop('ARIMA(0,0,0)', axis=1, inplace=True)
    
#   Set validation set timeline
    validation_test_timeline = get_consecutive_months(end_date_train)
    final_best_df = final_best_df[final_best_df['period'].isin(validation_test_timeline)]
    
    final_best_df_mape = final_best_df.iloc[:,:4]
    for cols in final_best_df.iloc[:,4:].columns:
        final_best_df_mape[cols] = final_best_df.apply(lambda row: calculate_mape(row['actual'], row[cols]), axis=1)
#   Check for bestfit config for each parts in ARIMA
    all_forecasted_data_mape_df = pd.DataFrame()
    for model_name in final_best_df_mape.columns[4:].tolist():
        print(model_name)
        model_mape_df = final_best_df_mape.groupby(['part_number'])[model_name].mean().reset_index()
        model_mape_df = model_mape_df.melt(id_vars='part_number', 
                                           var_name=model_name, 
                                           value_vars=model_name)
        model_mape_df.columns = ['part_number', 'model', 'value']
        all_forecasted_data_mape_df = pd.concat([all_forecasted_data_mape_df, model_mape_df], ignore_index=True)
    
    all_forecasted_data_mape_df_max = all_forecasted_data_mape_df.loc[all_forecasted_data_mape_df.groupby('part_number')['value'].idxmax()].reset_index(drop=True)
    
    final_df_best_fit_arima = pd.DataFrame()
    for pnt in all_forecasted_data_mape_df_max['part_number'].unique():
        best_model_name = all_forecasted_data_mape_df_max[all_forecasted_data_mape_df_max['part_number']==pnt]['model'].values[0]
        forecasted_data_subset_pnt = final_best_df.loc[final_best_df['part_number']==pnt, 
                                                       ['part_number', 'period', 
                                                        'actual', best_model_name]]
        forecasted_data_subset_pnt['model_name'] = best_model_name
        forecasted_data_subset_pnt.rename(columns={best_model_name:'forecasted_values'}, inplace=True)
        final_df_best_fit_arima = pd.concat([final_df_best_fit_arima, forecasted_data_subset_pnt], ignore_index=True)
    
    final_df_best_fit_arima = final_df_best_fit_arima.assign(accuracy_pred = 
                                                             lambda x: np.where(x['actual']!=0,
                                                                                (1-(abs(x['forecasted_values'] - x['actual'])/x['actual'])), 0))
    final_df_best_fit_arima['period'] = pd.to_datetime(final_df_best_fit_arima['period']).astype(str)
#   Extract bestfit ARIMA config for each parts - to be used during forecasting
    bestfit_arima_models_config = final_df_best_fit_arima[['part_number', 'model_name']].drop_duplicates().reset_index(drop=True)
    
    final_df_best_fit_arima['model_name'] = "ARIMA"
    final_df_best_fit_arima = final_df_best_fit_arima.rename(columns={'part_number':'Part Number',
                                                                            'model_name': 'Model Name',
                                                                            'actual': 'Actual',
                                                                            'forecasted_values': 'Forecasted',
                                                                            'period': 'Date',
                                                                            'accuracy_pred': 'New Model Accuracy'                    
                                                                           })
    final_df_best_fit_arima.loc[final_df_best_fit_arima['Forecasted']<0, 'Forecasted'] = 0
    final_df_best_fit_arima.loc[final_df_best_fit_arima['Forecasted']<0, 'New Model Accuracy'] = 0
    
    return final_df_best_fit_arima, bestfit_arima_models_config

In [ ]:
if negative_parts != []:
    arima_training_data, bestfit_config_arima = arima_training(negative_parts, data_old_lag1, 
                                                               training_start_date, training_end_date)
    prefix_arima_train_data_path = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{config.get_value('model_output_folder_name')}"
    prefix_arima_pickle = f"{config.get_value('primary_path')}/{config.get_value('tpcap_model')}/{config.get_value('model_training_date')}/{'arima'}"
    arima_train_data_file_name = config.get_value('arima_train_data_file_name')
    arima_bestfit_config_file_name = config.get_value('arima_bestfit_config_file_name')
    arima_training_data.to_excel(f"s3://{bucket}/{prefix_arima_train_data_path}/{arima_train_data_file_name}{pd.to_datetime(config.get_value('model_training_date')).strftime('%Y-%m-%d')}.xlsx", index=False)
    bestfit_config_arima.to_excel(f"s3://{bucket}/{prefix_arima_pickle}/{arima_bestfit_config_file_name}{pd.to_datetime(config.get_value('model_training_date')).strftime('%Y-%m-%d')}.xlsx", index=False)
else:
    pass

2330062030 0
-------------------Start-----------------------
1
Total data date:: 2021-07-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
47459BZ050 1
-------------------Start-----------------------
1
Total data date:: 2021-02-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
532860D380 2
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
7461002260B0 3
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
82210BZ020 4
-------------------Start-----------------------
1
Total data date:: 2022-05-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
---------

-------------------End-----------------------
477820K050 40
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
4806909071 41
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
482100K231 42
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
4851009Y40 43
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
485108Z175 44
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 

-------------------End-----------------------
61611BZ470 80
-------------------Start-----------------------
1
Total data date:: 2022-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
623320A010 81
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
67003BZ640 82
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
681050K430 83
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
68162YE010 84
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 

-------------------End-----------------------
842500K110C1 120
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
846520D081 121
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
871070K010 122
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
8713902100 123
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00:00:00 2024-04-01 00:00:00 2024-06-01 00:00:00
-------------------End-----------------------
8713902140 124
-------------------Start-----------------------
1
Total data date:: 2021-01-01 00:00:00 2024-03-01 00

#### ARIMA Forecasting

In [ ]:
def arima_model_forecast(data_ts, test_data, p_range, d_range, q_range, pn, train_x, valid_x):
    final_arima_df = pd.DataFrame()
    internal_arima_dict = pd.DataFrame()
    final_df = pd.DataFrame()
    model_list = []
    model = ARIMA(data_ts, order=(p_range, d_range, q_range))
    try: 
        fitted_model = model.fit()
        forecast_valid = fitted_model.forecast(steps=3)
        forecast_valid = forecast_valid.tolist()
        forecast_valid = list(map(lambda x: round(x, 2), forecast_valid))
        fitted_values = fitted_model.fittedvalues.tolist()
        fitted_values = list(map(lambda x: round(x, 2), fitted_values))
        model_name = f"ARIMA({p_range},{d_range},{q_range})"
        final_arima_df_test = pd.DataFrame({
            'part_number': [pn]*test_data.shape[0],
            'period':valid_x.values.flatten(),
            'type':['test']*test_data.shape[0],
            'actual':np.nan,
            'fitted':forecast_valid,
            'model_name':[model_name]*test_data.shape[0]})
        final_arima_df_train = pd.DataFrame({
            'part_number': [pn]*data_ts.shape[0],
            'period':train_x.values.flatten(),
            'type':['train']*data_ts.shape[0],
            'actual':data_ts.values,
            'fitted':fitted_values,
            'model_name':[model_name]*data_ts.shape[0]})
        final_df = pd.concat([final_df, final_arima_df_train, final_arima_df_test], ignore_index=True)
    except Exception as e:
        print(pn, " :: ", e)

    return final_df

In [ ]:
def create_TS_model_arima_forecast(df, p_range, d_range, q_range, t_start_date, t_end_date, lag_val):
    # Split data into training and testing sets
    pn = df['part_number'].unique().tolist()[0]
    df_new = df[:]

    train, test = create_complete_dataset(df_new, t_start_date, t_end_date)
    train_clean = train[:]
    if test.empty:
        print(pn)
        test = create_test_complete_dataset_from_scratch(test, 
                                                         train_clean['period'].max()+pd.DateOffset(months=1), 
                                                         train_clean['period'].max()+pd.DateOffset(months=3),
                                                         pn)
    if (test.shape[0]<3):
        test = create_test_complete_dataset(test, 
                                            train_clean['period'].max()+pd.DateOffset(months=1), 
                                            train_clean['period'].max()+pd.DateOffset(months=3))
    data_start_date = train['period'].min()
    train_end_date = train['period'].max()
    test_start_date = test['period'].min()
    data_end_date = test['period'].max()
    df_new['period'] = pd.to_datetime(df_new['period'])
    print("Total data date::", data_start_date, train_end_date, test_start_date, data_end_date)
    period_data = pd.concat([train_clean, test], ignore_index=True)
    print(train.shape, test.shape)
    arima_all_df = arima_model_forecast(train['Total_qty_per_month'][:], test[:], p_range, 
                                               d_range, q_range, pn, train['period'], test['period'])
    return arima_all_df

In [ ]:
def arima_forecast(part_list_neg, parts_config_df, clean_train_data, start_date_forecast, end_date_forecast):
    ind = 0
    arima_data_forecast = pd.DataFrame()

    parts_list_best_fit = part_list_neg[:]
    for parts in parts_list_best_fit:
        mod_name = parts_config_df[parts_config_df['part_number']==parts]['model_name'].values[0]
        p1,d1,q1 = list(map(lambda x: int(x),mod_name[6:-1].split(',')))
        print('-------------------Start-----------------------')
        for lag_val in range(1):
            if lag_val==0:
                parts_data_new = clean_train_data[clean_train_data['part_number']==parts].drop_duplicates()
                arima_df = create_TS_model_arima_forecast(parts_data_new, p1,d1,q1, start_date_forecast, end_date_forecast, lag_val)
                arima_data_forecast = pd.concat([arima_data_forecast, arima_df], ignore_index=True)
        ind+=1
        print('-------------------End-----------------------')
    arima_data_forecast = arima_data_forecast[arima_data_forecast['type']=='test'].reset_index(drop=True)
    arima_data_forecast['period'] = pd.to_datetime(arima_data_forecast['period']).astype(str)
    
    arima_data_forecast['model_name'] = "ARIMA"
    arima_data_forecast = arima_data_forecast.rename(columns={'part_number':'Part Number',
                                                                            'model_name': 'Model Name',
                                                                            'actual': 'Actual',
                                                                            'fitted': 'Forecasted',
                                                                            'period': 'Date'
                                                                           }).drop(['type'], axis=1)
    
    return arima_data_forecast

In [ ]:
forecast_end_date = (pd.to_datetime(data_old_lag1['period']).max().strftime('%Y-%m-%d'))

In [ ]:

if negative_parts != []:
    forecast_end_date = (pd.to_datetime(data_old_lag1['period']).max().strftime('%Y-%m-%d'))
    final_forecasted_arima_negative_data = arima_forecast(negative_parts, bestfit_config_arima, data_old_lag1, 
                                                          training_start_date, forecast_end_date)
    prefix_arima_forecasted_data_path = config.get_value('prefix_model_output_forecast_path')
    arima_forecasted_file_name = config.get_value('arima_forecasted_file_name')
    final_forecasted_arima_negative_data.to_excel(f"s3://{bucket}/{prefix_arima_forecasted_data_path}/{config.get_value('model_forecasting_date')}/{arima_forecasted_file_name}{pd.to_datetime(config.get_value('model_forecasting_date')).strftime('%Y-%m-%d')}.xlsx", index=False)
else:
    pass

### ARIMA process concluded

### PowerBI data setup

In [ ]:
bucket_curated = config.get_value('bucket_new')
prefix_gtmaster = config.get_value('prefix_gtmaster')
prefix_proc = config.get_value('prefix_proc')
prefix_pic_master_path = config.get_value("prefix_pic_master_path")
product_master_path_prefix = config.get_value('product_master_path_prefix')

In [ ]:
# Function to read a single Parquet file
def read_parquet_file(file):
    return pd.read_parquet(f's3://{file}', engine='pyarrow')

def gt_parquet_read(bucket_curated_name, gt_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{gt_master_parquet}/*/*/*/*.parquet')
    parquet_files_gt_partm = fs.glob(f's3://{bucket_curated_name}/{gt_master_parquet}*.parquet')

    df_master_gt_part_master = pd.DataFrame()
    for file in parquet_files_gt_partm:
        df_gt = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_gt_part_master = pd.concat([df_master_gt_part_master,df_gt],ignore_index=True)
    df_master_gt_part_master_subset = df_master_gt_part_master.sort_values('sales_start_dt', ascending=False)
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop_duplicates(subset=['part_no'], keep='first')
    df_master_gt_part_master_subset = df_master_gt_part_master_subset[["part_no", "country_origin_cd"]].drop_duplicates()
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.assign(IMPORT_CD = 
                                                                             lambda x: np.where(x['country_origin_cd']=='TH', 
                                                                                                'LSP', 
                                                                                                np.where(x['country_origin_cd']=='JP', 
                                                                                                         'JSP', 'MSP')))
    df_master_gt_part_master_subset = df_master_gt_part_master_subset.drop('country_origin_cd', axis=1)
    df_master_gt_part_master_subset.columns = [col.upper() for col in df_master_gt_part_master_subset.columns]
    return df_master_gt_part_master_subset

def proc_parquet_read(bucket_curated_name, proc_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{proc_master_parquet}/*/*/*/*.parquet')
    parquet_files_proc = fs.glob(f's3://{bucket_curated_name}/{proc_master_parquet}*.parquet')

    df_master_proc_master = pd.DataFrame()

    for file in parquet_files_proc:
    #     print(file)
        df_proc = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_proc_master = pd.concat([df_master_proc_master,df_proc],ignore_index=True)
    df_master_proc_master = df_master_proc_master.sort_values(['start_dt'], ascending=[False])
    df_master_proc_master = df_master_proc_master.drop_duplicates(subset='part_no', keep='first')
    df_master_proc_master_subset = df_master_proc_master[['part_no', 'procurement_div_cd', 'procurement_operator_id', 'inventory_ctrl_class']].drop_duplicates()
    df_master_proc_master_subset['procurement_div_cd'] = df_master_proc_master_subset['procurement_div_cd'].apply(lambda x:x[-3:])
    df_master_proc_master_subset = df_master_proc_master_subset.rename(columns={'procurement_div_cd':'Vendor Code',
                                                                                'part_no':'PART_NO',
                                                                                'procurement_operator_id':'PROCUREMENT_OPERATOR_ID',
                                                                                'inventory_ctrl_class': 'ICC_seg'
                                                                       })
    df_master_proc_master_subset['ICC_seg'] = df_master_proc_master_subset['ICC_seg'].apply(lambda x: x[0] if pd.notna(x) and len(x) > 1 else x)
    return df_master_proc_master_subset

def pic_parquet_read(bucket_curated_name, pic_master_parquet):
    # Recursively list all Parquet files in the nested S3 path
#     parquet_files_pic_master = fs.glob(f's3://{bucket_curated_name}/{pic_master_parquet}/*/*/*/*.parquet')
    parquet_files_pic_master = fs.glob(f's3://{bucket_curated_name}/{pic_master_parquet}*.parquet')


    df_master_pic = pd.DataFrame()

    for file in parquet_files_pic_master:
    #     print(file)
        df_pic = pd.read_parquet(f's3://{file}', engine='pyarrow')
        df_master_pic = pd.concat([df_master_pic,df_pic],ignore_index=True)
    
    df_master_pic_subset = df_master_pic[['id', 'pic', 'pic_abb']].drop_duplicates().reset_index(drop=True)
    df_master_pic_subset.columns = [col.upper() for col in df_master_pic_subset.columns]

    return df_master_pic_subset

def prod_master_read(bucket_curated_name, pg_master_path):
    pg_master_filepath = "s3://"+fs.glob(f's3://{bucket_curated_name}/{pg_master_path}/*.csv')[0]
    pg_master_df = read_file(pg_master_filepath)
    pg_master_df = pg_master_df[['PART_NO', 'PRODUCT_GROUP']].drop_duplicates()
    return pg_master_df

In [ ]:
def get_lastest_partition_path(bucket_name, prefix):
    if prefix[-1] != "/": prefix += "/"
    
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    year_month_day = {}

    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
            for obj in page.get('Contents', []):
                key = obj['Key']
                key = key[len(prefix):]
                parts = key.split('/')
#                 print(parts)
                if len(parts) >= 3:
                    if "=" in parts[0] and "=" in parts[1]:
                        year = parts[0].split('=')[-1]
                        month = parts[1].split('=')[-1]
                        if "=" in parts[2]:
                            day = parts[2].split('=')[-1]
                        else: day = ""
                            
                        if len(day) > 2: continue

                        if year not in year_month_day:
                            year_month_day[year] = {}
                        if month not in year_month_day[year]:
                            year_month_day[year][month] = set()
                        year_month_day[year][month].add(day)

#     print(year_month_day.keys())
    
    most_recent_year = max(year_month_day.keys(), key=int)
    most_recent_month = max(year_month_day[most_recent_year].keys(), key=int)
    if year_month_day[most_recent_year][most_recent_month] != {""}:
        most_recent_day = max(year_month_day[most_recent_year][most_recent_month], key=int)
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/day={most_recent_day}/"
    else:
        final_path = f"{prefix}year={most_recent_year}/month={most_recent_month}/"
        
    return final_path

In [ ]:
def create_final_master_file(bucket_cur, gt_parquet_path, proc_parquet_path, pic_parquet_path, prod_path):
    prefix_gtmaster = get_lastest_partition_path(bucket_cur, gt_parquet_path)
    gt_data = gt_parquet_read(bucket_cur, prefix_gtmaster)
    prefix_procmaster = get_lastest_partition_path(bucket_cur, proc_parquet_path)
    proc_data = proc_parquet_read(bucket_cur, prefix_procmaster)
    prefix_picmaster = get_lastest_partition_path(bucket_cur, pic_parquet_path)
    pic_data = pic_parquet_read(bucket_cur, prefix_picmaster)
    
    pg_data = prod_master_read(bucket_cur, prod_path)
    
    pic_data['ID'] = pic_data['ID'].astype(str)
    gt_data['PART_NO'] = gt_data['PART_NO'].astype(str)
    proc_data['PART_NO'] = proc_data['PART_NO'].astype(str)
    pg_data['PART_NO'] = pg_data['PART_NO'].astype(str)
    
    final_master_file_data = pd.merge(pd.merge(pd.merge(gt_data, proc_data, on='PART_NO', how='left'), 
                                 pic_data, left_on='PROCUREMENT_OPERATOR_ID', 
                                 right_on='ID', how='left').drop('ID', axis=1), 
                        pg_data, on='PART_NO', how='left')
    
    return final_master_file_data

In [ ]:
full_master_details = create_final_master_file(bucket_curated, prefix_gtmaster, prefix_proc, prefix_pic_master_path, product_master_path_prefix)

In [ ]:
full_master_details.shape, full_master_details['PART_NO'].nunique()

In [ ]:
def create_powerbi_data(arima_train_data, arima_forecasted_data, powerbi_data, parts_list_neg):
    powerbi_data_nonneg = powerbi_data[~powerbi_data['Part Number'].isin(parts_list_neg)]
    powerbi_data_neg = powerbi_data[powerbi_data['Part Number'].isin(parts_list_neg)]
    
    full_arima_data = pd.concat([arima_train_data, arima_forecasted_data], ignore_index=True)
    
    full_arima_data = normalize_forecast(full_arima_data, 'Forecasted')
    
    col_to_attach = powerbi_data_neg.drop(['Actual', 'Forecasted', 'Model Name','New Model Accuracy', 'Threshold'], axis=1).columns
    
    new_arima_powerbi_data = pd.merge(powerbi_data_neg[col_to_attach], 
                                      full_arima_data, on=['Part Number', 'Date'], how='left')
    
    new_total_powerbi = pd.concat([powerbi_data_nonneg, 
                                   new_arima_powerbi_data[powerbi_data_nonneg.columns]], ignore_index=True)
    new_total_powerbi['Actual'] = new_total_powerbi['Actual'].apply(lambda x: int(x) if pd.notna(x) else x)
    new_total_powerbi['Forecasted'] = new_total_powerbi['Forecasted'].apply(lambda x: int(x) if pd.notna(x) else x)
    return new_total_powerbi

In [ ]:
if negative_parts != []:
    training_grp = arima_training_data.groupby('Part Number').agg({'Actual':'sum', 'Forecasted':'sum'}).reset_index()

    training_grp['Closeness_to_accuracy'] = training_grp.apply(lambda row: 
                                                                 calculate_mape(row['Actual'], 
                                                                    row['Forecasted']), axis=1)

    training_grp['Avg_residual'] = (training_grp['Actual'] - training_grp['Forecasted'])/3
    training_grp = training_grp.assign(Variation = lambda x: np.where(np.logical_and(x['Avg_residual'] > -10, x['Avg_residual'] < 10), 1, 0))
    training_grp['Threshold'] = training_grp.apply(lambda x:create_threshold(x['Closeness_to_accuracy'], x['Variation']), axis=1)

    arima_training_data_new = arima_training_data.merge(training_grp[['Part Number', 'Threshold']].drop_duplicates(), on='Part Number', how='left')
    final_forecasted_arima_negative_data_new = final_forecasted_arima_negative_data.merge(training_grp[['Part Number', 'Threshold']].drop_duplicates(), on='Part Number', how='left')
else:
    pass

In [ ]:
if negative_parts != []:
    final_powerbi_data = create_powerbi_data(arima_training_data_new, final_forecasted_arima_negative_data_new, 
                                             total_powerbi_data, negative_parts)
else:
    final_powerbi_data = total_powerbi_data[:]

In [ ]:
old_col_names_pos = final_powerbi_data.columns

In [ ]:
full_master_details = full_master_details.rename(columns={'ICC_seg':'ICC',
                                                          'Vendor Code':'Supplier Name',
                                                          'PIC_ABB':'PIC (Abb)',
                                                          'PIC':'Person in Charge',
                                                          'PRODUCT_GROUP': 'Product Group',
                                                          'IMPORT_CD': 'Source Type'
                                                         })

In [ ]:
final_powerbi_data_sub = final_powerbi_data.drop(['ICC', 'Source Type', 'Product Group', 'Supplier Name', 'Person in Charge', 'PIC (Abb)'], axis=1)
final_powerbi_data_new = final_powerbi_data_sub.merge(full_master_details, left_on='Part Number', right_on='PART_NO', how='left').drop('PART_NO', axis=1)
final_powerbi_data_new = final_powerbi_data_new[old_col_names_pos]

In [ ]:
final_forecasted_file_path_prefix = config.get_value('prefix_forecast_file')
final_file_path_prefix = config.get_value('final_file_path_prefix')
final_file_path_bucket = config.get_value('bucket_new')
final_file_name = config.get_value('final_file_name')

In [ ]:
final_powerbi_data_new.to_csv(f"s3://{final_file_path_bucket}/{final_forecasted_file_path_prefix}{final_file_name}{pd.to_datetime(config.get_value('model_forecasting_date')).strftime('%Y-%m-%d')}.csv", index=False)

#### Add historical data to dashboard data

In [ ]:
def attach_historical_data(master_df, df, end_date_validation, end_date_training):
    test_months = get_consecutive_months(end_date_training)
    master_df = master_df.drop(columns="Count_of_order", axis=1)
    master_df = master_df.rename(columns={"Total_qty_per_month": "Actual"})
    master_df["period"] = pd.to_datetime(master_df["period"])
    df["Date"] = pd.to_datetime(df["Date"])
    df_hist = df[df['Date']<=end_date_validation]
    df_future = df[df['Date']>end_date_validation]
    df_hist["Date"] = pd.to_datetime(df_hist["Date"])
    if master_df['part_number'].nunique()>df_hist['Part Number'].nunique():
        master_df=master_df[master_df['part_number'].isin(df_hist['Part Number'].unique())]
    else:
        master_df=master_df[:]
    
    merged_df = pd.merge(master_df, df_hist, left_on=["part_number", "period"], right_on=["Part Number", "Date"], how="left")
    merged_df = merged_df.sort_values(by=["part_number", "period"])
    merged_df.drop(columns=["Part Number", "Date"], axis=1, inplace=True)
    merged_df.drop(columns=["Actual_y"], axis=1, inplace=True)
    merged_df.rename(columns={"Actual_x": "Actual"}, inplace=True)

    columns_to_bfill = ['Model Name', 'ICC', 'Product Group', 'Source Type', 'Lag', 
                        'Supplier Name', 'Person in Charge', 'PIC (Abb)', 'Threshold', 
                        'Forecast_flag']
    merged_df = merged_df.sort_values(['part_number', 'period'])

    merged_df['Year'] = merged_df['period'].dt.year

    merged_df['Month Name'] = merged_df['period'].dt.month_name()

    merged_df['Month'] = merged_df['period'].dt.month

    merged_df['sort'] = merged_df['period'].dt.year.astype(str) + merged_df['period'].dt.month.astype(str)

    merged_df['months'] = merged_df['period'].dt.strftime('%b').astype(str) + merged_df['period'].dt.strftime('%y').astype(str)

    merged_df.rename(columns={"period": "Date"}, inplace=True)

    merged_df.rename(columns={"part_number": "Part Number"}, inplace=True)

    merged_df = merged_df[['Part Number', 'Model Name', 'ICC', 'Date', 'Actual', 'Forecasted', 'Old Model Accuracy',
        'New Model Accuracy', 'Product Group', 'Source Type', 'Lag', 'Supplier Name', 'Person in Charge', 'PIC (Abb)',
        'Threshold', 'Year', 'Month Name', 'Month', 'sort', 'months','Forecast_flag'
    ]]
    
    merged_df_future = pd.concat([merged_df, df_future], ignore_index=True)
    
    merged_df_future.loc[merged_df_future['Date'].isin(test_months), 'Forecast_flag'] = 'History_dash'
    
    merged_df_future['Actual'] = merged_df_future['Actual'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    
    
    columns_to_fill = ['Model Name', 'ICC', 'Product Group', 'Source Type', 'Lag', 
                        'Supplier Name', 'Person in Charge', 'PIC (Abb)', 'Threshold', 'Forecast_flag']
    # Vectorized fill operation
    merged_df_future[columns_to_fill] = merged_df_future.groupby('Part Number')[columns_to_fill] \
                                           .apply(lambda group: group.bfill().ffill()) \
                                           .reset_index(level=0, drop=True)

    merged_df_future = merged_df_future.rename(columns={'Threshold':'Model Selection Criteria'})
    return merged_df_future

In [ ]:
# final_powerbi_data.shape

In [ ]:
final_dashboard_data_withhistory = attach_historical_data(data_old_lag1, final_powerbi_data_new, forecast_end_date,training_end_date)

In [ ]:
final_dashboard_data_withhistory.columns

In [ ]:
final_dashboard_data_withhistory = final_dashboard_data_withhistory.sort_values(by=['Part Number', 'Date'])

In [ ]:
# Backfill columns A, B, C where D is 'F' and the values are NaN
final_dashboard_data_withhistory.loc[final_dashboard_data_withhistory['Forecast_flag'] == 'Future', ['Supplier Name', 'Person in Charge', 'PIC (Abb)']] = final_dashboard_data_withhistory.loc[final_dashboard_data_withhistory['Forecast_flag'] == 'Future', ['Supplier Name', 'Person in Charge', 'PIC (Abb)']].ffill()

In [ ]:
final_file_with_history_name = config.get_value('final_file_with_history_name')

In [ ]:
archive_path = config.get_value('final_file_with_history_archive_prefix')

In [ ]:
archive_path

In [ ]:
def get_last_month_date():
    # Get the current date
    current_date = datetime.now()

    # Subtract one month and set the day to 1
    first_date_previous_month = current_date.replace(day=1) - relativedelta(months=1)
    first_date_previous_month = first_date_previous_month.replace(day=1)

    # Format the date in YYYY-MM-DD
    formatted_date = first_date_previous_month.strftime('%Y-%m-%d')

    return formatted_date

Data Continuity - Get old data and add new months' updated data

for example - 
- last run has data trained till Jun-2024 and forecasted from Jul-Sep 2024
- for new run, saved models will capture data till Jul-2024 and forecast for Aug-Oct 2024 data; 
- during the process, the report will be having old Jul data with no actuals as per algorithm
- the below process updates Jul data with actual value and new master data details

In [ ]:
# Check if dashboard data exists in the folder: 
# if yes - continue adding new month data to current dashboard data; 
# if no - its a new run

# Specify your S3 path
s3_path = f"s3://{final_file_path_bucket}/{final_file_path_prefix}/{final_file_with_history_name[:-1]}.csv"

# Check if the file exists
files = fs.glob(s3_path)

if files:
    # Previous month dashboard data
    old_dash_data = read_file("s3://"+files[0])
    old_dash_data['Date'] = pd.to_datetime(old_dash_data['Date'], format='%Y-%m-%d')
    # get last month date
    last_month_date = get_last_month_date()

    # Clean data
    data_old_lag1['period'] = pd.to_datetime(data_old_lag1['period'], format='%Y-%m-%d')
    
    # new month data
    dash_data_middle_months = old_dash_data[old_dash_data['Date']==last_month_date]
    # Update new month data with actual values
    dash_data_middle_months_merge = dash_data_middle_months.merge(data_old_lag1.drop('Count_of_order', axis=1), 
                                                              left_on=['Part Number', 
                                                                       'Date'], 
                                                              right_on=['part_number', 
                                                                        'period'], 
                                                              how='left').drop(['period', 
                                                                                'part_number', 
                                                                                'Actual'], 
                                                                               axis=1).rename(columns={'Total_qty_per_month':'Actual'})
    dash_data_middle_months_merge = dash_data_middle_months_merge[dash_data_middle_months.columns]
    dash_data_middle_months_merge['Actual'] = dash_data_middle_months_merge['Actual'].apply(lambda x: int(x) if pd.notna(x) else x)
    final_dashboard_data_withhistory = final_dashboard_data_withhistory[final_dashboard_data_withhistory['Date']!=last_month_date].reset_index(drop=True)
    # extract master data details from new dashboard data
    new_full_data_pn_model = final_dashboard_data_withhistory[['Part Number', 'Model Name', 'Product Group',
           'Source Type', 'Lag', 'Supplier Name', 'Person in Charge', 'PIC (Abb)',
           'Model Selection Criteria']].drop_duplicates()
    # merge master data details to new data
    dash_data_middle_months_merge = dash_data_middle_months_merge.drop(['Model Name', 'Product Group',
       'Source Type', 'Lag', 'Supplier Name', 'Person in Charge', 'PIC (Abb)',
       'Model Selection Criteria'], axis=1).merge(new_full_data_pn_model, on='Part Number')
    dash_data_middle_months_merge = dash_data_middle_months_merge[dash_data_middle_months.columns]
    dash_data_middle_months_merge['New Model Accuracy'] = dash_data_middle_months_merge.apply(lambda row: 
                                                                                          calculate_mape(row['Actual'], 
                                                                                                         row['Forecasted']), axis=1)
#     dash_data_middle_months_merge.loc[dash_data_middle_months_merge['Date']==last_month_date, 'Forecast_flag'] = 'History_dash'
    # update new months details with master data to new dashboard data
    new_full_data_complete = pd.concat([final_dashboard_data_withhistory, 
                                        dash_data_middle_months_merge], ignore_index=True).sort_values(['Part Number', 'Date'])
    
    # Update flags
    forecasted_bestfit_data['Date'] = pd.to_datetime(forecasted_bestfit_data['Date'])
    start_date = pd.to_datetime(forecasted_bestfit_data['Date'].min().strftime("%Y-%m-%d"))
    historic_dash_month_list = [(start_date - pd.DateOffset(months=i)).strftime('%Y-%m-%d') for i in range(1, 4)]
    # get list of new historic dash months
    new_full_data_complete.loc[new_full_data_complete['Date'].isin(historic_dash_month_list), 
                               'Forecast_flag'] = 'History_dash'
    
    forecasted_bestfit_data['Date'] = pd.to_datetime(forecasted_bestfit_data['Date']).dt.strftime('%Y-%m-%d')
    forecasted_month_list = forecasted_bestfit_data['Date'].unique().tolist()
    
    months_list = forecasted_month_list+historic_dash_month_list
    
    # Update rest months to 'History' tagged
    new_full_data_complete.loc[~new_full_data_complete['Date'].isin(months_list), 'Forecast_flag'] = 'History'
    new_full_data_complete['Forecasted'] = new_full_data_complete['Forecasted'].apply(lambda x: np.where(x<0,max(x,0),x))
    
    # Changing for Athena table
    new_full_data_complete['Forecasted'] = new_full_data_complete['Forecasted'].astype(float)
    new_full_data_complete['Actual'] = new_full_data_complete['Actual'].astype(float)
    new_full_data_complete['Old Model Accuracy'] = new_full_data_complete['Old Model Accuracy'].astype(float)
    new_full_data_complete['New Model Accuracy'] = new_full_data_complete['New Model Accuracy'].astype(float)
    # save data to s3 folder
    # Below line saves data in archive folder
    new_full_data_complete.to_csv(f"s3://{final_file_path_bucket}/{archive_path}/{final_file_with_history_name}{pd.to_datetime(config.get_value('model_forecasting_date')).strftime('%Y-%m-%d')}.csv", index=False)
    new_full_data_complete.to_csv(f"s3://{final_file_path_bucket}/{final_file_path_prefix}/{final_file_with_history_name[:-1]}.csv", index=False)
else:
    # Changing for Athena table
    final_dashboard_data_withhistory['Forecasted'] = final_dashboard_data_withhistory['Forecasted'].astype(float)
    final_dashboard_data_withhistory['Actual'] = final_dashboard_data_withhistory['Actual'].astype(float)
    final_dashboard_data_withhistory['Old Model Accuracy'] = final_dashboard_data_withhistory['Old Model Accuracy'].astype(float)
    final_dashboard_data_withhistory['New Model Accuracy'] = final_dashboard_data_withhistory['New Model Accuracy'].astype(float)
    # Below line saves data in archive folder
    final_dashboard_data_withhistory.to_csv(f"s3://{final_file_path_bucket}/{archive_path}/{final_file_with_history_name}{pd.to_datetime(config.get_value('model_forecasting_date')).strftime('%Y-%m-%d')}.csv", index=False)
    final_dashboard_data_withhistory.to_csv(f"s3://{final_file_path_bucket}/{final_file_path_prefix}/{final_file_with_history_name[:-1]}.csv", index=False)